In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Precio_Municipios.csv')

In [3]:
df.head()

,Entidad,Municipio,CP,Year,Num_Transacciones,Precio_x_m2
0,Guanajuato,Salamanca,36887,2020,1,9066.79
1,Guanajuato,Salamanca,36885,2020,16,9452.25
2,Guanajuato,Salamanca,36883,2020,1,8543.76
3,Guanajuato,Salamanca,36877,2020,1,12610.97
4,Guanajuato,Salamanca,36875,2020,1,11820.10


In [4]:
df.Year.unique()

array([2020, 2019, 2018, 2017, 2016, 2015], dtype=int64)

In [5]:
precios = df[(df['Year'] == 2015) | (df['Year'] == 2020) ][['Entidad', 'Municipio', 'CP', 'Year', 'Precio_x_m2']]
precios = pd.pivot_table(precios, values = 'Precio_x_m2', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()
precios.rename(columns = {2015: 'precio_15', 2020: 'precio_20'}, inplace = True)
precios['plus_20/15'] = precios['precio_20'] / precios['precio_15']
precios.head()

Year,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15
0,Guanajuato,Salamanca,36700,6954.65,9370.61,1.347388
1,Guanajuato,Salamanca,36710,8375.76,10517.59,1.255718
2,Guanajuato,Salamanca,36714,5571.65,NaN,NaN
3,Guanajuato,Salamanca,36715,9976.22,NaN,NaN
4,Guanajuato,Salamanca,36717,10479.00,13433.10,1.281907


In [6]:
min15 =  precios['precio_15'].min()
min16 =  precios['precio_20'].min()
minpl =  precios['plus_20/15'].min()

precios['precio_15'] = precios['precio_15'].fillna(min15) 
precios['precio_20'] = precios['precio_20'].fillna(min16)
precios['plus_20/15'] = precios['plus_20/15'].fillna(minpl)

In [ ]:
precios['precio_15'] = precios['precio_15'].fillna(0) 
precios['precio_20'] = precios['precio_20'].fillna(0)
precios['plus_20/15'] = precios['plus_20/15'].fillna(0)

In [110]:
precios[ precios['precio_15'].isnull()].reset_index()

Year,index,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15
0,6,Guanajuato,Salamanca,36722,NaN,7896.52,NaN
1,19,Guanajuato,Salamanca,36759,NaN,7591.67,NaN
2,29,Guanajuato,Salamanca,36777,NaN,8233.83,NaN
3,35,Guanajuato,Salamanca,36786,NaN,8542.84,NaN
4,40,Guanajuato,Salamanca,36863,NaN,14939.65,NaN
5,42,Guanajuato,Salamanca,36870,NaN,10725.68,NaN
6,44,Guanajuato,Salamanca,36875,NaN,11820.10,NaN
7,45,Guanajuato,Salamanca,36877,NaN,12610.97,NaN
8,49,Guanajuato,Salamanca,36887,NaN,9066.79,NaN
9,108,México,Coacalco de Berriozábal,55705,NaN,12205.71,NaN


In [130]:
precios[(precios['precio_15'].notnull())  & (precios['Municipio']== 'Salamanca')]['precio_15'].quantile([.2, .4, .6, .8]).values

array([5986.01, 6456.37, 7521.51, 8375.76])

In [136]:
consulta = precios[precios['Municipio'] == 'Salamanca'][['precio_15']].reset_index()
qs = consulta[consulta['precio_15'].notnull()]['precio_15'].quantile([.2, .4, .6, .8]).values
qs

array([5986.01, 6456.37, 7521.51, 8375.76])

In [7]:
variables = ['precio_15', 'precio_20', 'plus_20/15']
alcaldia = precios.Municipio.unique()

for w in variables:
    nombre = "Cal_"+str(w)
    precios[nombre] = ""


for x in alcaldia:   
    for y in variables:
        
        consulta = precios[precios['Municipio'] == str(x)][[str(y)]].reset_index()
        qs = consulta[y].quantile([.2, .4, .6, .8]).values
        nombre = "Cal_"+str(y)
    
        for a,b in consulta.itertuples(index=False):
            if b <= qs[0]:
                precios.loc[a, str(nombre)] = "6"
            elif (b <= qs[1]) & (b > qs[0]):
                precios.loc[a, str(nombre)] = "7"
            elif (b <= qs[2]) & (b > qs[1]):
                precios.loc[a, str(nombre)] = "8"
            elif (b <= qs[3]) & (b > qs[2]):
                precios.loc[a, str(nombre)] = "9"
            else:
                precios.loc[a, str(nombre)] = "10"

In [65]:
precios

Year,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15,Cal_precio_15,Cal_precio_20,Cal_plus_20/15
0,Guanajuato,Salamanca,36700,6954.65,9370.61,1.347388,8,8,9
1,Guanajuato,Salamanca,36710,8375.76,10517.59,1.255718,9,9,7
2,Guanajuato,Salamanca,36714,5571.65,NaN,NaN,6,10,10
3,Guanajuato,Salamanca,36715,9976.22,NaN,NaN,10,10,10
4,Guanajuato,Salamanca,36717,10479.00,13433.10,1.281907,10,10,8
...,...,...,...,...,...,...,...,...,...
141,Puebla,Cuautlancingo,72724,8329.33,12946.02,1.554269,6,9,10
142,Puebla,Cuautlancingo,72725,11195.41,NaN,NaN,10,10,10
143,Puebla,Cuautlancingo,72730,9835.08,11867.63,1.206663,10,8,6
144,Puebla,Cuautlancingo,72733,9234.51,11225.56,1.215610,9,7,7


In [8]:
precios

Year,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15,Cal_precio_15,Cal_precio_20,Cal_plus_20/15
0,Guanajuato,Salamanca,36700,6954.65,9370.61,1.347388,8,9,9
1,Guanajuato,Salamanca,36710,8375.76,10517.59,1.255718,10,9,8
2,Guanajuato,Salamanca,36714,5571.65,5346.28,0.623941,7,6,6
3,Guanajuato,Salamanca,36715,9976.22,5346.28,0.623941,10,6,6
4,Guanajuato,Salamanca,36717,10479.00,13433.10,1.281907,10,10,9
...,...,...,...,...,...,...,...,...,...
141,Puebla,Cuautlancingo,72724,8329.33,12946.02,1.554269,6,10,10
142,Puebla,Cuautlancingo,72725,11195.41,5346.28,0.623941,10,6,6
143,Puebla,Cuautlancingo,72730,9835.08,11867.63,1.206663,10,8,7
144,Puebla,Cuautlancingo,72733,9234.51,11225.56,1.215610,9,7,7


In [58]:
transacciones = df[(df['Year'] == 2015) | (df['Year'] == 2016) | (df['Year'] == 2017) | (df['Year'] == 2018) | (df['Year'] == 2019)][['Entidad', 'Municipio', 'CP', 'Year', 'Num_Transacciones']]
transacciones = pd.pivot_table(transacciones, values = 'Num_Transacciones', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()
transacciones.rename(columns = {2015: 'tran_15', 2016: 'tran_16', 2017: 'tran_17', 2018: 'tran_18', 2019: 'tran_19'}, inplace = True)
transacciones.head()

Year,Entidad,Municipio,CP,tran_15,tran_16,tran_17,tran_18,tran_19
0,Guanajuato,Salamanca,36700,13.0,28.0,29.0,18.0,23.0
1,Guanajuato,Salamanca,36704,NaN,NaN,NaN,NaN,1.0
2,Guanajuato,Salamanca,36710,161.0,39.0,122.0,6.0,NaN
3,Guanajuato,Salamanca,36714,2.0,5.0,2.0,1.0,2.0
4,Guanajuato,Salamanca,36715,1.0,NaN,1.0,NaN,NaN
